In [1]:
import os
# ggf muss Pfad angepasst werden
os.chdir("{}/..".format(os.getcwd()))
os.getcwd()


'/home/lukas/Projects/dmc2019'

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection, linear_model, metrics
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split, cross_validate
#from scripts.utils import own_scorer, calc_scores

## Data

In [3]:
df_train = pd.read_csv("data/extended_train.csv", sep="|")
sum_frauds, sum_non_frauds  = len(df_train[df_train.fraud == 1]), len(df_train[df_train.fraud == 0])
df_y = df_train.fraud
df_X = df_train.drop(['fraud'], axis=1)

train_x, val_x, train_y, val_y = train_test_split(df_X, df_y, test_size=0.2)

print("Shape of train data: ", train_x.shape, "Shape of val data: ", val_x.shape)
df_train.head()

Shape of train data:  (1503, 10) Shape of val data:  (376, 10)


,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud,totalScannedLineItems
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0,29.0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0,14.0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0,13.0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0,29.0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0,27.0


## Data with new Features

In [ ]:
df = df_train
df['totalScannedLineItems'] = df['scannedLineItemsPerSecond'] * df['totalScanTimeInSeconds']
df['avgTimePerScan'] = 1/ df['scannedLineItemsPerSecond']
df['avgValuePerScan'] = df['avgTimePerScan'] * df['valuePerSecond']
df['withoutRegisPerPosition'] = df['scansWithoutRegistration'] / df['totalScannedLineItems'] #equivalent to lineItemVoidsPerPosition?
df['quantiModPerPosition'] = df['quantityModifications'] / df['totalScannedLineItems']
df['lineItemVoidsPerTotal'] = df['lineItemVoids'] / df['grandTotal']
df['withoutRegisPerTotal'] = df['scansWithoutRegistration'] / df['grandTotal']
df['quantiModPerTotal'] = df['quantityModifications'] / df['grandTotal']
df['lineItemVoidsPerTime'] = df['lineItemVoids'] / df['totalScanTimeInSeconds']
df['withoutRegisPerTime'] = df['scansWithoutRegistration'] / df['totalScanTimeInSeconds']
df['quantiModPerTime'] = df['quantityModifications'] / df['totalScanTimeInSeconds']
df['valuePerScannedLineItem'] = df['valuePerSecond'] / df['scannedLineItemsPerSecond']
df1_y = df.fraud
df1_X = df.drop(['fraud'], axis=1)

In [4]:
print("Frauds: ", sum_frauds, "Non Frauds: ", sum_non_frauds)

Frauds:  104 Non Frauds:  1775


### Scoring Methods

In [5]:
scoring = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172)}

In [6]:
params = {
    "max_depth": randint(2, 6),  # default 3
    "n_estimators": randint(300, 450),  # default 100
    "eta" :  uniform(0.1, 0.7),
    "gamma" : uniform(0,2),
    "min_child_weight" : uniform(0,50),
    "max_delta_step" : uniform(0,10), #Set it to value of 1-10 might help control the update.
    "reg_lambda" : uniform(0.1,2),
    "scale_pos_weight" : [1, sum_non_frauds/sum_frauds],
    "max_bin" : randint(200, 300)
}
default_xgb = xgb.XGBClassifier(booster="gbtree",tree_method='gpu_hist', disable_default_eval_metric=1,objective='binary:logistic',eval_metric='aucpr', n_jobs=-1, verbosity=2)
search = RandomizedSearchCV(default_xgb, scoring=scoring, param_distributions=params, random_state=42, n_iter=10000,
                            cv=3, verbose=1, n_jobs=-1, return_train_score=True,refit='FBeta')
search.fit(df_X, df_y)
results = search.cv_results_

Fitting 3 folds for each of 10000 candidates, totalling 30000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   30.0s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  8.0min


KeyboardInterrupt: 

In [ ]:
scorings = {"DMC" : own_scorer, "DMC_Norm" : own_scorer_normalized}
xgbo = search.best_estimator_
res = cross_validate(xgbo, df_X, df_y, scoring=scorings, cv=5, n_jobs=-1)

In [ ]:
res

In [ ]:
results

In [ ]:
search.best_estimator_